# Branch and Bound

Bei der systematischen Suche nach einer Lösung in einer sehr großen Menge von Kandidaten ist eine vollständige Breiten- oder Tiefensuche praktisch meist nicht mehr möglich. Man stelle sich z.B. ein 0/1 Rucksackproblem mit 50 Gegenständen vor. Bei diesem gibt es über eine Billiarde potentielle Belegungen, die nicht alle geprüft werden können. Eine vollständige Traversierung des Suchbaums ist nicht mehr möglich. Hier ist eine andere Herangehensweise gefragt.

<img src="img/BnB_FullSearchTree.png" width="500">

Wie von der Tiefensuche bekannt ist, kann die Menge der Kandidaten (alle Blätter im Suchbaum, repräsentiert durch die Wurzel) systematisch in kleinere Teilmengen (Blätter unterhalb eines Knotenzweigs) unterteilt werden. Durch dieses **Verzweigen** (branch) entstehen Teilmengen, die jeweils unterschiedliche spezielle Eigenschaften aufweisen. Die Menge der Kandidaten bei o.g. Rucksackproblem kann z.B. in die Teilmenge A (alle Rucksackbelegungen wo Gegenstand 1 enthalten ist) und die Teilmenge B (alle Rucksackbelegungen wo Gegenstand 1 nicht enthalten ist) verzweigt werden. Diese Mengen können nun wieder verzweigt bzw. in noch kleinere Teilmengen unterteilt werden bezüglich des Enthaltens der anderen Gegenstände. So entsteht ein typischer vollständiger Suchbaum.

Anstatt den Suchbaum vollständig zu durchlaufen, kann man sich die Frage stellen, welche Knoten es überhaupt wert sind weiter  verfolgt zu werden. Möglicherweise ist die Wahrscheinlichkeit die Lösung unterhalb eines bestimmten Knotens zu finden größer als bei einem anderen. Vielleicht ist es sogar unmöglich, dass sich die Lösung unterhalb eines bestimmten Knotens befindet. In diesem Fall kann durch **Begrenzen** (bound) eine Expandierung solcher Knoten unterbunden werden.

Die generelle Vorgehensweise beim Entwickeln eines Branch and Bound Algorithmus beinhaltet zunächst den **Entwurf des Suchbaums**. Da am Ende nur die Blätter (die möglichen Lösungen) interessant sind, kann der Baum oberhalb dieser Blätter beliebig gestaltet werden. Des Weiteren wird eine **Bewertungsfunktion** benötigt. Diese muss for einen Knoten entscheiden, ob dieser expandiert werden soll oder vernachlässigt werden kann. Zusätzlich sollte eine **Reihenfolge** festgelegt werden in der die Knoten verarbeitet werden sollen. Zur effektiven Begrenzung ist es notwendig möglichst schnell zu guten Teillösungen zu gelangen um dadurch enge Schranken für die Bewertungsfunktion zu erhalten.

Sind diese drei Eigenschaften optimal aufeinander abgestimmt, kann die Suche nach einer Lösung sehr viel effizienter werden. Dies ist aber nicht zwangsläufig der Fall. Können z.B. keine Knoten ausgeschlossen werden, dann wird praktisch eine vollständige Suche durchgeführt, die durch die zusätzliche Bewertung am Ende langsamer ist als z.B. eine einfache Tiefensuche. Oder wenn die Bewertung ineffizient ist, könnte der dadurch entstehende Verlust den Gewinn durch die Begrenzung aufheben oder gar übersteigen.

## Entwurf des Algorithmus

Für ein 0/1 Rucksackproblem mit 4 Gegenständen könnte genau der oben gezeigte Suchbaum entstehen. Auf jeder Ebene wird für einen bestimmten Gegenstand die Entscheidung getroffen, ob dieser im Rucksack enthalten sein soll oder nicht. Wurden 4 Entscheidungen getroffen, steht eine mögliche Rucksackbelegung fest. Diese kann nun die Lösung sein oder auch nicht.

Um diesen Baum zu begrenzen, müssen Möglichkeiten gefunden werden bereits für einen Knoten zu entscheiden ob die Lösung unter diesem zu finden ist. Beim Rucksackproblem bietet die Kapazität des Rucksacks eine solche.

<img src="img/BnB_TreeBoundStatic.png" width="500">

Für vier Gegenstände mit den Gewichten 5,4,3,2 und einer Maximalkapazität von 8 können die Zweige ignoriert werden, bei denen die Kapazitätsgrenze überschritten wird (grau). Von 16 möglichen Kandidaten bleiben noch 10 übrig und anstatt 15 Expansionen vorzunehmen sind nur noch 9 erforderlich.

Die Kapazitätsgrenze ist eine statische Schranke denn sie ändert sich während der Suche nicht. Besser sind immer **dynamische Schranken** die im Verlauf der Suche immer strenger werden. Wurde z.B. bereits irgendeine gültige Rucksackbelegung gefunden, können Knoten unter denen nur schlechtere Belegungen zu finden sind ignoriert werden. Je besser der aktuell beste Rucksack im Verlauf der Suche wird, umso strenger wird die Schranke und umso stärker fällt die Begrenzung aus. Das Ziel ist also möglichst schnell zu einer guten (noch nicht optimalen) Lösung zu gelangen um von einer möglichst starken Begrenzung zu profitieren.

Gibt es z.B. bereits einen gültigen Rucksack mit einem gewissen Wert so müsste für einen Knoten (quasi einen noch nicht fertig gefüllten Rucksack) bestimmt werden, ob überhaupt noch ein gültiger Rucksack zusammengestellt werden kann, der wertvoller ist. An jedem Knoten hat der untersuchte Teil-Rucksack einen momentanen Wert und eine Restkapazität. Um zu bestimmen, welcher zusätzliche Wert unter Verbrauch der Restkapazität noch hinzugefügt werden kann, müsste allerdings ein weiteres (kleineres) Rucksackproblem gelöst werden. **Die Effizienz einer Bewertungsfunktion ist aber entscheidend.** Diese Effizienz kann dadurch erreicht werden, dass eine weniger genaue Schranke ermittelt wird. Für das 0/1 Rucksackproblem bietet sich z.B. folgende Möglichkeit:

Die $n$ Gegenstände mit den Werten $w_i$ und den Gewichten $g_i$ für $1 <= i <= n$ können nach ihrem spezifischen Wert $\frac{w_i}{g_i}$ sortiert entschieden werden. Dieser Quotient sagt aus, welcher Wert pro Gewichtseinheit beim Hinzufügen dieses Gegenstands hinzugewonnen wird. Entscheidungen für Gegenstände mit hohem Wert und geringem Gewicht werden so zuerst getroffen. Es ist sichergestellt, dass $\frac{w_i}{g_i} >= \frac{w_{i+1}}{g_{i+1}}$. Für einen Knoten (einen Teil-Rucksack) bei dem bereits $k$ Gegenstände entschieden wurden, kann bei gegebener Maximalkapazität $K$ die Restkapazität $r$ und der damit einhergehende maximal erreichbare Rucksackwert $m$ bestimmt werden.
(Ein (Teil-)Rucksack ist definiert durch den Vektor $\overrightarrow{x}=(x_1, x_2, \dots, x_k) \text{ für } 1 <= k <= n \text{ und } x_i \in {0,1}$. Der Rucksack $(1,0,1)$ enthält demnach Gegenstand 1 und 3 und nicht Gegenstand 2.)
$$\\
r = K - \sum_{i=1}^k x_i g_i\\
m = r \frac{w_{k+1}}{g_{k+1}} + \sum_{i=1}^k x_i w_i
$$
Die Restkapazität $r$ eines Rucksack ist die Differenz zwischen Maximalkapazität und aktuellem Rucksackgewicht und der maximal erreichbare Rucksackwert $m$ ist die Summe aus dem aktuellem Rucksackwert und der Restkapazität multipliziert mit dem spezifischen Wert des nächsten noch nicht entschiedenen Gegenstands. Liegt der Wert des momentan besten Rucksacks bereits über diesem maximal erreichbaren Wert, so kann der entsprechende Knoten ignoriert werden.

<img src="img/BnB_TreeBoundRest.png" width="600">

Der mögliche Wertezugewinn kann so für jeden Knoten sehr einfach berechnet werden. Obiges Bild zeigt die entsprechenden Zugewinne an den Knoten für die rechts aufgelisteten Gegenstände. Wie effektiv die resultierende Begrenzung ist, hängt nun von der Reihenfolge ab in der die Knoten geprüft werden. In jedem Fall muss bei Knoten <font color='red'>A</font> begonnen werden. Bei <font color='red'>A</font> ist der aktuelle Rucksack leer (hat den Wert $0$) und es ist ein Zugewinn von maximal $14,4$ zu erwarten. Da $0+14,4$ größer ist als der momentan beste Rucksack (ist zu Beginn ebenfalls leer) muss <font color='red'>A</font> expandiert werden. Durch die Expansion wird ein Rucksack gefunden der Gegenstand 1 enthält und somit den Wert $9$ hat. Dieser wird nun zum momentan besten bekannten Rucksack und gibt mit seinem Wert eine erste Schranke vor. Als nächste Knoten kommen somit <font color='red'>B</font> und <font color='red'>C</font> in Frage. Bei <font color='red'>B</font> ist der aktuelle Rucksackwert $9$ und durch den Zugewinn von $5$ kann ein maximaler Rucksackwert von $14$ erreicht werden. Bei <font color='red'>C</font> ist der maximal zu erreichende Wert nur $13,3$. Aus diesem Grund sollte Knoten <font color='red'>B</font> bevorzugt behandelt werden. Folgende Tabelle zeigt die weitere Entwicklung:

| Schritt | Knoten | Rucksackwert | Zugewinn | Max | Aktion | Best |
| --- | --- | --- | --- | --- | --- | --- |
| 1 | A | 0 | 14,4 | 14,4 | Expandiere zu B und C | 9 |
| 2 | B | 9 | 5 | 14 | Expandiere zu D und E | 14 |
|   | C | 0 | 13,3 | 13,3 | Aufschieben |  |
| 3 | C | 0 | 13,3 | 13,3 | Bound (Max $\leq$ Best) |  |
|   | D | 14 | 0 | 14 | Gehe zu H | 14 |
|   | E | 9 | 4,5 | 13,5 | Bound (Max $\leq$ Best) |  |
| 4  | H | 14 | 0 | 14 | Ende (keine Knoten mehr) | 14 |

Die Expansion von <font color='red'>B</font> in Schritt 2 führt zu einem neuen besten Rucksack wodurch die Bound-Schranke strenger wird. In Schritt 3 kann für die verfügbaren Knoten <font color='red'>C</font> und <font color='red'>E</font> festgestellt werden, dass sich unterhalb keine bessere Lösung befinden kann. Damit ergibt sich folgender dynamisch reduzierter Suchbaum bei dem weitere 6 Expansionen eingespart wurden.

<img src="img/BnB_TreeBoundResult.png" width="600">


## Implementation 0/1 Rucksackproblem mittels Branch and Bound

Nur selten werden Branch and Bound Algorithmen rekursiv implementiert. Eine sortierte Queue ist zumeist die beste Option um eine Menge abzuarbeitender Elemente zu verwalten, so wie es hier für die Knoten erforderlich ist. Die meisten höheren Programmiersprachen bieten hierzu entsprechende Datenstrukturen an. Hier wird *heapq* verwendet um die zur Expansion zur Verfügung stehenden Knoten nach ihrem maximal erreichbaren Rucksackwert sortiert bereitzustellen. Die Auswahl des als nächstes zu expandierenden Knotens kann so durch ein einfaches *heappop* realisiert werden.

Die Expansion, also das Erzeugen neuer Knoten mit und ohne dem nächsten Gegenstand, erfolgt schlicht durch anhängen von 1 oder 0 and die Liste welche einen Rucksack repräsentiert. Die expandierten Knoten werden nur dann der Queue hinzugefügt wenn die Kapazitätsgrenze nicht überschritten wird und der maximal erreichbare Wert den momentan besten Rucksackwert überschreitet.

Gibt es keine zu prüfenden Knoten mehr, also wurden alle Knoten entweder bereits verarbeitet oder ausgeschlossen, steht das Endergebnis fest.

In [4]:
import heapq # Priority Queue
K = 8
allitems = [(5, 9), (3, 5), (4, 6), (2,3)]

def bnbKnapsack(items, K):
    # Sortierung der Gegenstände nach Gewicht pro Werteinheit (kleiner ist besser)
    sortedItems = sorted(items, key=lambda x: x[0] / x[1])
    bestKS = []  # [0,1,1,0] heißt das item1 und 2 enthalten sind und item0 und 3 nicht, anfangs leer
    # Hilfsfunktion zur Bestimmung des Rucksackwerts
    def ksValue(ks): return sum([sortedItems[i][1] if ks[i] == 1 else 0 for i in range(len(ks))])
    # Hilfsfunktion zur Bestimmung des Rucksackgewichts
    def ksWeight(ks): return sum([sortedItems[i][0] if ks[i] == 1 else 0 for i in range(len(ks))])
    # Hilfsfunktion zur Bestimmung des maximal erreichbaren Rucksackwerts
    def limit(ks): return ksValue(ks)+(K-ksWeight(ks))*sortedItems[len(ks)][1]/sortedItems[len(ks)][0]

    knapsacks = []  # die aktuellen Knoten, sortiert nach maximal erreichbarem Rucksackwert
    heapq.heappush(knapsacks, (1 / limit([]), []))  # Wurzel ist der leere Rucksack
    while len(knapsacks): # solange es Knoten zu prüfen gibt
        ks = heapq.heappop(knapsacks)[1]  # Knoten mit dem höchsten erreichbaren Rucksackwert als nächstes
        print("check:", ks)
        for x in [0,1]: # Expandieren zu 2 neuen Knoten
            expanded = ks + [x]
            print("expanded:", expanded, end=" ")
            if ksWeight(expanded) <= K:  # Kapazitätsgrenze noch nicht überschritten
                best = ksValue(bestKS) # momentan bester Rucksack
                if len(expanded) < len(sortedItems):  # noch kein vollständiger Rucksack
                    m = limit(expanded) # maximal zu erreichender Wert
                    if m > best:  # könnte ein besserer Rucksack erzielt werden?
                        heapq.heappush(knapsacks, (1 / m, expanded))
                        print("queued with max:", m)
                    else:
                        print("bound")
                if best < ksValue(expanded):  # besser?
                    bestKS = expanded
                    print(expanded, "better")
            else:
                print("too heavy")
        
    return bestKS  


print("Best:",bnbKnapsack(allitems, K))
        

check: []
expanded: [0] queued with max: 13.333333333333334
expanded: [1] queued with max: 14.0
[1] better
check: [1]
expanded: [1, 0] queued with max: 13.5
expanded: [1, 1] queued with max: 14.0
[1, 1] better
check: [1, 1]
expanded: [1, 1, 0] bound
expanded: [1, 1, 1] too heavy
check: [1, 0]
expanded: [1, 0, 0] bound
expanded: [1, 0, 1] too heavy
check: [0]
expanded: [0, 0] bound
expanded: [0, 1] bound
Best: [1, 1]
